In [ ]:
import yfinance as yf
btc_ticker = yf.Ticker("BTC-USD")
btc_data = btc_ticker.history(period="max")

In [ ]:
btc_data

In [ ]:
from matplotlib import pyplot as plt
plt.plot(btc_data.Close)
plt.show()

In [ ]:
btc_data["moving_average"] = btc_data["Close"].rolling(10).mean()


In [ ]:
btc_data[['Close','moving_average']].plot()


In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(btc_data["Close"][9:], btc_data["moving_average"][9:])


In [ ]:
dataset_x = []
dataset_y = []
for j in range(0, 30):
    for i in range(j, btc_data.shape[0], 31):
        if btc_data[i : i + 31].Close.shape[0] == 31:
            dataset_y.append(
                1
                if btc_data[i : i + 30].Close.mean() < btc_data.iloc[i + 30].Close
                else 0
            )
        mean_ = btc_data[i : i + 30].Close.mean()
        std_ = btc_data[i : i + 30].Close.std()
        dataset_x.append(list((btc_data[i : i + 30].Close - mean_) / std_))


In [ ]:
import numpy as np
dataset_x = np.array(dataset_x)
dataset_y = np.array(dataset_y)
And respectively, create train and test sets:
train_x, test_x = dataset_x[:3000], dataset_x[3000:]
train_y, test_y = dataset_y[:3000], dataset_y[3000:]

In [ ]:
def encoder(inputs, head_size, num_heads, feed_forward_dimension, dropout=0):
    # Attention layer
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    attention_result = x + inputs
    # Convlution as feedforward
    x = layers.Conv1D(filters=feed_forward_dimension, kernel_size=1, activation="relu")(
        attention_result
    )
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + attention_result


In [ ]:
def build_model(
    input_size,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    dense_units,
    dropout=0,
    dense_dropout=0,
):
    inputs = keras.Input(shape=input_size)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = encoder(x, head_size, num_heads, ff_dim, dropout)
        x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for d in dense_units:
        x = layers.Dense(d, activation="relu")(x)
        x = layers.Dropout(dense_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)


In [ ]:
model = build_model(
    x_train.shape[1:],
    head_size=256,
    num_heads=4,
    ff_dim=4,
    dropout=0.25,
    num_transformer_blocks=4,
    dense_units=[128],
    dense_dropout=0.4,
)


In [ ]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["sparse_categorical_accuracy"],
)
model.summary()


In [ ]:
callbacks = [keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]


In [ ]:
model.fit(
    x_train,
    y_train,
    epochs=5,
    batch_size=64,
    callbacks=callbacks,
)


In [ ]:
model.evaluate(x_test, y_test, verbose=1)